# Pivotal Greenplum-Spark Connector
## PySpark Example

----

Pivotal Greenplum-Spark Connector documentation (notes below extracted from Pivotal documentation):

https://greenplum-spark.docs.pivotal.io/110/index.html

----

Steps to launching Jupyter Notebook with Greenplum-Spark connector available

1. Download greenplum-spark connector from Pivotal network https://network.pivotal.io/products/pivotal-gpdb (version used for this example greenplum-spark_2.11-1.1.0.jar)

2. Set environment variables - pyspark will launch Jupyter Notebook
```bash
# set environment variables
export PYSPARK_DRIVER_PYTHON='ipython'
export PYSPARK_DRIVER_PYTHON_OPTS='notebook --port=8888 --no-browser --ip=0.0.0.0 --notebook_dir=/notebooks'
```
3. Launch Jupyter Notebook
```bash
# Launch notebooks
# Set jar to location of greenplum-spark connector jar
pyspark --master spark://spark:7077 --jars=../spark-jars/greenplum-spark_2.11-1.1.0.jar
```

*Note - Wine data set used in example https://archive.ics.uci.edu/ml/datasets/wine*

----

#### Import Wine data set for example

In [1]:
# load wine dataset for example
import psycopg2

connString = "host='192.168.99.100' dbname='gpadmin' user='gpadmin' password='pivotal' port=5432"
conn = psycopg2.connect(connString)
conn.autocommit = True
cur = conn.cursor()

# create external web table
query = """
    DROP EXTERNAL TABLE IF EXISTS public.wine_external;
    CREATE EXTERNAL WEB TABLE public.wine_external (
         cultivars integer
        ,alcohol float
        ,malic_acid float
        ,ash float
        ,alcalinity_of_ash float
        ,magnesium float
        ,total_phenols float
        ,flavanoids float
        ,nonflavanoid_phenols float
        ,proanthocyanins float
        ,color_intensity float
        ,hue float
        ,od280_od315 float
        ,proline integer
    ) LOCATION ('http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data')
    FORMAT 'CSV';
    
    DROP TABLE IF EXISTS public.wine;
    CREATE TABLE public.wine AS 
    SELECT * 
    FROM public.wine_external
    DISTRIBUTED BY (cultivars);
    
"""
cur.execute(query)

#### PySpark Example

In [2]:
# dependencies
import pyspark              # http://spark.apache.org/docs/latest/api/python/
#from pyspark.sql import SQLContext

In [11]:
from pyspark import SparkContext

In [12]:
from pyspark import SQLContext

In [14]:
from pyspark.conf import SparkConf

In [15]:
 spark = SparkSession.builder \
     .master("local") \
     .appName("Word Count") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()


Note that the .load() operation does not initiate the movement of data from Greenplum Database to Spark. 
Spark employs lazy evaluation for transformations; it does not compute the results until the application 
performs an action on the DataFrame, such as displaying or filtering the data or counting the number of rows.

https://greenplum-spark.docs.pivotal.io/110/read_from_gpdb.html

Options
* **url** format jdbc:postgresql://[hostname]:[port]/[database]
* **dbtable** table must be in GPDB search_path and have a distribution column (can not be distributed randomly)
* **partitionColumn** must be of type in [bigint, bigserial, integer, serial]

In [4]:
spark.
# create pointer to table 'pivotal.testing' in greenplum
#sqlContext = SQLContext(sc)
gpdf = sqlContext.read.format("io.pivotal.greenplum.spark.GreenplumRelationProvider").options(
    url="jdbc:postgresql://192.168.99.100:5432/gpadmin",
    user="gpadmin",
    password="pivotal",
    dbtable="wine",
    partitionColumn="cultivars").load()


SyntaxError: invalid syntax (<ipython-input-4-23f4ccc37448>, line 1)

In [6]:
gpdf = pyspark.sql.read("io.pivotal.greenplum.spark.GreenplumRelationProvider").options(
    url="jdbc:postgresql://192.168.99.100:5432/gpadmin",
    user="gpadmin",
    password="pivotal",
    dbtable="wine",
    partitionColumn="cultivars").load()


AttributeError: 'module' object has no attribute 'read'

Note: By default, Spark recomputes a transformed DataFrame each time you run an action on it. 
If you have a large data set on which you want to perform multiple transformations, you may choose 
to keep the DataFrame in memory for performance reasons. You can use the DataSet.persist() method 
for this purpose. Keep in mind that there are memory implications to persisting large data sets.

In [ ]:
gpdf.persist()

In [ ]:
# Check out data types of columns
gpdf.printSchema()

In [ ]:
# Column names 
gpdf.columns

In [ ]:
# row count
gpdf.count()

In [ ]:
# show first 5 rows
gpdf.show(5, truncate=True)

In [ ]:
# summary stats
# toPandas(): pySpark dataframe -> pandas dataframe
gpdf.describe().toPandas()

In [ ]:
# select a subset of columns
gpdf.select(gpdf.columns[0:2]).show(5)

In [ ]:
# Select first 5 columns, filter results to where cultivars = 1 and show top 5 when ranked by alcohol

# select columns -> filter rows -> order results by
gpdf.select(gpdf.columns[0:5]).filter("cultivars = 1").orderBy("alcohol").limit(5).toPandas()

**Running Spark SQL query against DataFrame**

In [ ]:
# Prepare temp table view for running SQL queries
gpdf.createGlobalTempView("wine")


In [ ]:
# Select first 5 columns, filter results to where cultivars = 1 and show top 5 when ranked by alcohol

# prepare query
query = """
    SELECT {} 
    FROM global_temp.wine 
    WHERE cultivars = 1
    ORDER BY alcohol
""".format(','.join(gpdf.columns[0:5]))

# run query
spark.sql(query).limit(5).toPandas()